In [18]:
%load_ext autoreload

In [173]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
%autoreload 2
from utils import join_target_api,join_to_sql,prepare_dataset
% matplotlib inline 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [197]:
api_data = pd.read_csv('../data/api_data_clean.csv',index_col='sk_id_curr')

In [198]:
sql_data = pd.read_csv('../data/processed/full_sql.csv',index_col='SK_ID_CURR')
sql_data = sql_data.drop(columns='TARGET')

In [199]:
train_target = pd.read_csv('../data/processed/train_target.csv',index_col='SK_ID_CURR')

In [200]:
test_target = pd.read_csv('../data/processed/test_target.csv',index_col='SK_ID_CURR')

In [201]:
train = join_target_api(train_target,api_data)
test = join_target_api(test_target,api_data)

In [202]:
train = join_to_sql(train,sql_data)
test = join_to_sql(test,sql_data)
train.head()

,TARGET,days_birth,flag_document_13,flag_document_9,name_education_type,name_income_type,obs_30_cnt_social_circle,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,BASEMENTAREA_AVG,ENTRANCES_AVG,FLAG_DOCUMENT_20,FONDKAPREMONT_MODE,CNT_CHILDREN,FLAG_DOCUMENT_15,FLAG_EMAIL,OWN_CAR_AGE
SK_ID_CURR,,,,,,,,,,,,,,,,,
100006,0.0,-19005,False,False,Secondary / secondary special,Working,2,0.0,-617.0,NaN,NaN,0,NaN,0,0,0,NaN
100014,0.0,-10197,False,False,Higher education,Working,0,0.0,-844.0,NaN,NaN,0,NaN,1,0,0,NaN
100015,0.0,-20417,False,False,Secondary / secondary special,Pensioner,0,0.0,-2396.0,NaN,NaN,0,NaN,0,0,0,NaN
100017,0.0,-14086,False,False,Secondary / secondary special,Working,0,0.0,-4.0,0.0973,0.1379,0,reg oper account,1,0,0,23.0
100018,0.0,-14583,False,False,Secondary / secondary special,Working,0,0.0,-188.0,0.1335,0.1724,0,reg oper account,0,0,0,NaN


In [203]:
train = prepare_dataset(train)
test = prepare_dataset(test)

In [204]:
train.dtypes

TARGET                      float64
days_birth                    int64
flag_document_13              int64
flag_document_9               int64
obs_30_cnt_social_circle      int64
DAYS_LAST_PHONE_CHANGE      float64
FLAG_DOCUMENT_20              int64
CNT_CHILDREN                  int64
FLAG_DOCUMENT_15              int64
FLAG_EMAIL                    int64
dtype: object

In [205]:
x_train,x_test,y_train,y_test = train_test_split(train.drop(columns='TARGET'),
                                                 train['TARGET'],
                                                 random_state = 0
                                                )

In [206]:
clf = RandomForestClassifier(n_estimators=100)

In [207]:
clf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [209]:
results = clf.predict_proba(x_test)[:,1]

In [210]:
roc_auc_score(y_test,results)

0.5386190603925921

In [214]:
results_test = clf.predict_proba(test.drop(columns=['TARGET']))[:,1]

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [215]:
test['prob'] = results_test

In [219]:
test['prob'].to_csv('first_submission.csv')